<a href="https://colab.research.google.com/github/DineshIT/DineshIT/blob/main/notebooks/gradio_demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Re-Aging Demos with Gradio
To try out the Gradio Demos using Google Colab, run the cells below.
Be sure to select a runtime with GPU (e.g. `Runtime > Change Runtime Type > T4 (GPU)`) to make the model run fast.

You can choose to either run the regular inference demo or the video demo. The demo will open in a new tab.

## Downloading files and model

In [17]:
!git clone https://github.com/timroelofs123/face_reaging

Cloning into 'face_reaging'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 139 (delta 23), reused 21 (delta 10), pack-reused 88 (from 1)
Receiving objects: 100% (139/139), 48.26 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [18]:
!git clone https://huggingface.co/timroelofs123/face_re-aging

fatal: destination path 'face_re-aging' already exists and is not an empty directory.


In [19]:
%mv face_re-aging/best_unet_model.pth face_reaging/
%cd face_reaging/

/content/face_reaging/face_reaging/face_reaging


## Installing requirements

In [20]:
%pip install gradio face_recognition antialiased_cnns kaleido av --quiet

## Running demo

Re-age your photo or video with the three demos.

In [21]:
import gradio as gr
import torch
from model.models import UNet
from scripts.test_functions import process_image, process_video

In [22]:
# default settings
window_size = 512
stride = 256
steps = 18
frame_count = 100
model_path = 'best_unet_model.pth'

In [23]:
# The code from this block equivalent to the "run" function in scripts.gradio_demo

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(model_path, map_location=device))
unet_model.eval()

def block_img(image):
    source_age = 25
    target_age = random.randint(65, 80)
    return process_image(unet_model, image, video=False, source_age=source_age,
                          target_age=target_age, window_size=window_size, stride=stride)

def block_img_vid(image):
    source_age = 25
    target_age = random.randint(65, 80)
    return process_image(unet_model, image, video=True, source_age=source_age,
                          target_age=target_age, window_size=window_size, stride=stride, steps=steps)

def block_vid(video_path):
    source_age = 25
    target_age = random.randint(65, 80)
    return process_video(unet_model, video_path, source_age, target_age,
                          window_size=window_size, stride=stride, frame_count=frame_count)

demo_img = gr.Interface(
    fn=block_img,
    inputs=[gr.Image(type="pil")],  # Removed sliders
    outputs="image",
    examples=[
        ['assets/gradio_example_images/1.png'],
        ['assets/gradio_example_images/2.png'],
        ['assets/gradio_example_images/3.png'],
        ['assets/gradio_example_images/4.png'],
        ['assets/gradio_example_images/5.png'],
        ['assets/gradio_example_images/6.png']
    ],
    description="Input an image of a person, and they will be aged to a randomly selected age between 65 and 80."
)

demo_img_vid = gr.Interface(
    fn=block_img_vid,
    inputs=[gr.Image(type="pil")],  # Removed slider
    outputs=gr.Video(),
    examples=[
        ['assets/gradio_example_images/1.png'],
        ['assets/gradio_example_images/2.png'],
        ['assets/gradio_example_images/3.png'],
        ['assets/gradio_example_images/4.png'],
        ['assets/gradio_example_images/5.png'],
        ['assets/gradio_example_images/6.png']
    ],
    description="Input an image of a person, and a video will be returned showing the aging effect."
)

demo_vid = gr.Interface(
    fn=block_vid,
    inputs=[gr.Video()],  # Removed sliders
    outputs=gr.Video(),
    examples=[
        ['assets/gradio_example_images/orig.mp4'],
    ],
    description="Input a video of a person, and it will be aged frame-by-frame."
)

demo = gr.TabbedInterface([demo_img, demo_img_vid, demo_vid],
                          tab_names=['Image inference demo', 'Image animation demo', 'Video inference demo'],
                          title="Face Re-Aging Demo",
                          )


<ipython-input-23-d9c828614266>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet_model.load_state_dict(torch.load(model_path, map_location=device))


In [25]:
demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98f835f282379ce10c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
